# Import

In [2]:
import pandas as pd
import numpy as np

# Funzioni

In [21]:
# definiamo una funzione che prende in input il nome di un driver e ritorna la somma dei punti e il numero di podi
def drivers_performance(driver_name):
    '''
    Funzione che prende in input il nome di un driver e ritorna:
    - La somma dei punti
    - Il numero di podi (prime 3 posizioni)
    - Il numero di vittorie (1ª posizione)

    Arg.:
    - driver_name: Nome del driver da analizzare (stringa)

    Rit.:
    - total_points: Somma totale dei punti del driver
    - total_podiums: Numero di podi (posizioni 1, 2, 3)
    - total_wins: Numero di vittorie (posizione 1)
    '''
    # Filtra i dati per il driver specificato
    if driver_name not in dataset['Driver'].unique():
        print(f"Driver '{driver_name}' not found in the dataset.")
        return None
    driver_data = dataset[dataset['Driver'] == driver_name]

    # Calcola le metriche richieste
    total_points = driver_data['Points'].sum()  # Somma dei punti
    total_podiums = driver_data['Position'].isin([1, 2, 3]).sum()  # Podi
    total_wins = (driver_data['Position'] == 1).sum()  # Vittorie

    # Ritorna un dizionario con i risultati
    results = {
        'total_points': total_points,
        'total_podiums': total_podiums,
        'total_wins': total_wins
    }

    return results



In [34]:
def ranking_constructors(df):
    '''
    Funzione per creare una classifica finale dei costruttori basata sulla somma totale dei punti.

    Arg:
    - df: DataFrame contenente i dati dei costruttori, incluse colonne 'Driver' e 'Points'.

    Rit:
    - Un DataFrame con la classifica dei costruttori in ordine decrescente di punti.
    '''

    # Calcolo della somma dei punti per ciascun pilota
    rank = (
        df.groupby('Team', as_index=False)['Points']
        .sum()
        .sort_values(by='Points', ascending=False)
        .reset_index(drop=True)
    )

    # Aggiunta della posizione di classifica
    rank['Rank'] = rank.index + 1

    return rank

In [25]:
def ranking_drivers(df):
    '''
    Funzione per creare una classifica finale dei piloti basata sulla somma totale dei punti.

    Arg:
    - df: DataFrame contenente i dati dei piloti, incluse colonne 'Driver' e 'Points'.

    Rit:
    - Un DataFrame con la classifica dei piloti in ordine decrescente di punti.
    '''

    # Calcolo della somma dei punti per ciascun pilota
    rank = (
        df.groupby('Driver', as_index=False)['Points']
        .sum()
        .sort_values(by='Points', ascending=False)
        .reset_index(drop=True)
    )

    # Aggiunta della posizione di classifica
    rank['Rank'] = rank.index + 1

    return rank


# Svolgimento

In [3]:
BASE_URL = "https://raw.githubusercontent.com/MarcoRagosta/Repository/refs/heads/main/"
dataset = pd.read_csv(BASE_URL+'formula1_data.csv')
dataset.head()

,Driver,Team,Race,Country,Position
0,Hamilton,McLaren,Melbourne,Australia,1
1,Massa,Ferrari,Melbourne,Australia,0
2,Raikkonen,Ferrari,Melbourne,Australia,8
3,Kubica,BMW,Melbourne,Australia,0
4,Alonso,Renault,Melbourne,Australia,4


#### Creiamo la colonna Points

In [7]:
# Aggiungiamo una nuova colonna 'Points' in base alla posizione
dataset['Points'] = np.select(
    condlist=[
        dataset['Position'] == 1,
        dataset['Position'] == 2,
        dataset['Position'] == 3,
        dataset['Position'] == 4,
        dataset['Position'] == 5,
        dataset['Position'] == 6,
        dataset['Position'] == 7,
        dataset['Position'] == 8
    ],
    choicelist=[10, 8, 6, 5, 4, 3, 2, 1],
    default=0
)

# Visualizzare le prime righe
dataset.head()

,Driver,Team,Race,Country,Position,Points
0,Hamilton,McLaren,Melbourne,Australia,1,10
1,Massa,Ferrari,Melbourne,Australia,0,0
2,Raikkonen,Ferrari,Melbourne,Australia,8,1
3,Kubica,BMW,Melbourne,Australia,0,0
4,Alonso,Renault,Melbourne,Australia,4,5


#### Analisi dei piloti

In [23]:
drivers_performance('Hamilton')

{'total_points': 98, 'total_podiums': 10, 'total_wins': 5}

In [26]:
ranking_drivers(dataset)

,Driver,Points,Rank
0,Hamilton,98,1
1,Massa,97,2
2,Kubica,75,3
3,Raikkonen,75,4
4,Alonso,61,5
5,Heidfeld,60,6
6,Kovalainen,53,7
7,Vettel,35,8
8,Trulli,31,9
9,Glock,25,10


#### Creiamo la classifica dei piloti

In [33]:
# Generiamo la classifica
ranking = ranking_drivers(dataset)

# Diamo il nome al file
file_name = "Drivers_Standings_2008.txt"

# Apriamo il file in modalità scrittura
with open(file_name, "w") as file:

    # Scriviamo il titolo
    file.write("=== Drivers Standings 2008 ===\n\n")

    # Scriviamo la classifica
    for _, row in ranking.iterrows():
        file.write(f"Rank {row['Rank']:>2}: {row['Driver']:<20} - Points: {row['Points']}\n")

    # Scrivo il mio nome
    file.write("\n--- By Marco Ragosta ---")

# Confermiamo la classifica
print(f"Classifica salvata con successo in {filename}")

Classifica salvata con successo in Drivers_Standings_2008.txt


#### Creiamo il ranking dei costruttori

In [35]:
ranking_constructors(dataset)

,Team,Points,Rank
0,Ferrari,172,1
1,McLaren,151,2
2,BMW,135,3
3,Renault,61,4
4,Toyota,56,5
5,Toro Rosso,35,6
